In [1]:
from multiprocessing import Process, Manager
import json
import os
from pprint import pprint
import re
from tqdm import tqdm
import random

import uuid
import openai
import tiktoken
import json
import numpy as np
import requests
from retry import retry
from scipy import sparse
#from rank_bm25 import BM25Okapi
#import jieba
from http import HTTPStatus
import dashscope
from dotenv import dotenv_values
config = dotenv_values('.env')

In [2]:
MODEL_NAME = 'qwen2-7b-instruct'
dashscope.api_key = config['qwen_key'],

@retry(delay=60, tries=3)
def call_qwen_api(MODEL_NAME, query):
    messages = [
        {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        MODEL_NAME,
        messages=messages,
        result_format='message',  # set the result is message format.
    )
    if response.status_code == HTTPStatus.OK:
        #print(response)
        return response['output']['choices'][0]['message']['content']
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        raise Exception()

In [3]:
def get_prompt(problem, question, options):

    options = '\n'.join(f"{'ABCDEFG'[i]}. {o}" for i, o in enumerate(options))

    prompt = f"""你是一个逻辑推理专家，擅长解决逻辑推理问题。以下是一个逻辑推理的题目，形式为单项选择题。所有的问题都是（close-world assumption）闭世界假设，即未观测事实都为假。请逐步分析问题并在最后一行输出答案，最后一行的格式为"答案是：A"。题目如下：

### 题目:
{problem}

### 问题:
{question}
{options}
"""

    return prompt

In [4]:
def extract(input_text):
    ans_pattern = re.compile(r"答案是：(.*)", re.S)

    # 使用 findall 方法找到所有匹配的字符串
    problems = ans_pattern.findall(input_text)

    # 提取答案中出现的所有可能的答案字母
    possible_answers = "ABCDEFG"  # 假设可能的答案字母是 A, B, C, D
    answers = [match for match in problems[0] if match in possible_answers]

#     print(answers)  # 输出：['A']
#     ans_pattern = re.compile(r"答案是：(.)", re.S)
#     problems = ans_pattern.findall(input_text)
    if len(answers) == 0:
        return ''
        
    return answers[0]

In [5]:
def produce(data, MODEL_NAME, return_list, pid):
    tqdm1 = tqdm
    for task in tqdm1(data):
            problem = task['problem']
            for question in task['questions']:

                prompt = get_prompt(problem, 
                                    question['question'],
                                    question['options'],
                                    )

                response = call_qwen_api(MODEL_NAME, prompt)
                try:
                    attemp = 1
                    while attemp <= 3:
                        #print(f'try：{attemp}')
                        extract_response = extract(response)
                        question[MODEL_NAME] = extract_response
                        if question[MODEL_NAME] == '':
                            print(f'wtrong response：{response}')
                            attemp += 1
                            continue
                        else:
                            attemp = 4
                    # if pid == 0:
                    #     pprint(extract_response)
                    # break
                except:
                    pass
            return_list.append(task)

In [6]:
def main(data, ofn):
    # 如果输出文件已经存在，则不进行处理
    # if os.path.exists(ofn):
    #     pass

    # 设置进程池大小
    POOL_SIZE = 5
    #data = []

    # 将数据分割成POOL_SIZE个子列表
    datas = [data[i::POOL_SIZE] for i in range(POOL_SIZE)]

    # 使用Manager创建一个共享对象
    with Manager() as manager:
        # 创建一个共享列表，用于存储所有进程的结果
        return_list = manager.list()

        # 创建进程列表
        producers = []
        for i in range(POOL_SIZE):
            # 创建每个进程，传递数据子集、模型名称、结果列表和进程编号
            p = Process(target=produce,
                        args=(datas[i],  # 数据子集
                              MODEL_NAME,  # 模型名称
                              return_list,  # 共享结果列表
                              i,  # 进程编号
                              )
                        )
            producers.append(p)

        # 启动所有进程
        for p in producers:
            p.start()

        # 等待所有进程完成
        for p in producers:
            p.join()

        # 输出结果列表的长度
        print(len(return_list))

        # 写入输出文件
        with open(ofn, 'w') as writer:
            for sample in return_list:
                # 将结果写入文件
                writer.write(json.dumps(sample, ensure_ascii=False))
                writer.write('\n')

    # 输出任务完成的消息
    print("All tasks finished!")

    # 调用评估函数
    evaluate(ofn)

In [7]:
def evaluate(ofn):
    data = []
    with open(ofn) as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

    pse = 0
    cnt = 0
    tot = 0
    for task in data:
        for question in task['questions']:
            
            if MODEL_NAME in question:
                tot += 1
                cnt += question[MODEL_NAME] == question['answer']
            else:
                pse += 1

    print(cnt, tot, cnt/tot, pse)

In [8]:
def read_file(ifn):
    # 读取输入文件
    data=[]
    with open(ifn) as reader:
        for line in reader:
            # 解析每一行JSON数据
            sample = json.loads(line)
            data.append(sample)

    # 打乱数据顺序
    #random.shuffle(data)
    return data

In [ ]:
if __name__ == '__main__':

    a = extract("""根据欧几里得算法，逐步解析计算两个数6和7的最大公约数（gcd）的步骤如下：

1. 判断6和7是否相等：不相等。
2. 判断6和7大小关系，7 > 6，所以用更大的数7减去较小的数6得到结果1。
3. 现在计算6和1的最大公约数。
4. 6 > 1，根据算法用更大的数6减去较小的数1得到结果5。
5. 再计算5和1的最大公约数。
6. 5 > 1，用5减去1得到结果4。
7. 再计算4和1的最大公约数。
8. 4 > 1，用4减去1得到结果3。
9. 再计算3和1的最大公约数。
10. 3 > 1，用3减去1得到结果2。
11. 再计算2和1的最大公约数。
12. 2 > 1，用2减去1得到结果1。
13. 最后计算1和1的最大公约数，两数相等，gcd即为这两个数，也就是1。

因此，6和7的最大公约数是1。

答案是：应该是阿飞.""")

    data = read_file('data/round1_train_data.jsonl')
    main(data, 'result/qwen.jsonl')

  8%|▊         | 8/100 [00:33<07:01,  4.58s/it]

Request id: 183bc60e-c1cc-952d-89ff-b4111a65f54f, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: ac4e254c-9465-9b30-b76a-c4ee7e2b3c91, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: 4930b560-2954-9fe3-aad5-ab98d7dbcbb6, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: 7f14a2f4-e4be-9b3f-9b27-57bd13b19898, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: 784df39b-49fa-9052-9ff8-660e1b8b9d9b, Status code: 400, error code: DataInspectionFailed, error message: Output data may contain inappropriate content.


  6%|▌         | 6/100 [01:45<46:20, 29.58s/it]t]

Request id: 14ec3c7b-e055-98c3-ae35-7486b829cb03, Status code: 400, error code: DataInspectionFailed, error message: Output data may contain inappropriate content.


 13%|█▎        | 13/100 [01:59<12:27,  8.59s/it] 

...

wtrong response：分析已知条件：
1. Justin 喜欢 Charlotte。
2. Miranda 喜欢 Justin。
3. Miranda 还喜欢 Bob。
4. Miranda 和 Charlotte 都喜欢 Kylee。
5. Kylee 喜欢 Miranda。

根据第4条和第5条，我们可以得知：
- Miranda 喜欢 Kylee。
- Kylee 喜欢 Miranda。

因为 Justin 喜欢 Charlotte（与他性别相同的女孩），并且根据已知条件，没有信息表明 Justin 喜欢 Kylee 或者对 Kylee 有其他情感关系。同时，Justin 对 Kylee 的嫉妒需要基于他喜欢的人（Charlotte）也喜欢 Kylee 这一前提。然而，题目中没有提供足够的信息来确定 Justin 是否会因为 Charlotte 也喜欢 Kylee 而感到嫉妒。因此，基于给定的信息，我们不能断定 Justin 是否会因为这个原因而感到嫉妒。

答案是：无法判断。
wtrong response：分析已知条件：
1. Justin 喜欢 Charlotte。
2. Miranda 喜欢 Justin。
3. Miranda 还喜欢 Bob。
4. Miranda 和 Charlotte 都喜欢 Kylee。
5. Kylee 喜欢 Miranda。

根据第4条和第5条，我们可以得知：
- Miranda 喜欢 Kylee。
- Kylee 喜欢 Miranda。

因为 Justin 喜欢 Charlotte（与他性别相同的女孩），并且根据已知条件，没有信息表明 Justin 喜欢 Kylee 或者对 Kylee 有其他情感关系。同时，Justin 对 Kylee 的嫉妒需要基于他喜欢的人（Charlotte）也喜欢 Kylee 这一前提。然而，题目中没有提供足够的信息来确定 Justin 是否会因为 Charlotte 也喜欢 Kylee 而感到嫉妒。因此，基于给定的信息，我们不能断定 Justin 是否会因为这个原因而感到嫉妒。

答案是：无法判断。
wtrong response：分析已知条件：
1. Justin 喜欢 Charlotte。
2. Miranda 喜欢 Justin。
3. Miranda 还喜欢 Bob。


 47%|████▋     | 47/100 [10:47<06:36,  7.49s/it]

Request id: 3e4c07e1-0b14-917e-bb10-a0c566da578d, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: 7f2d5ac6-3b14-9a6e-b7db-69dd67a5de64, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


 41%|████      | 41/100 [10:49<19:13, 19.56s/it]

Request id: 953d27c8-c60c-99bc-aeab-e3a81de19f14, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.
Request id: 4a524b6f-0493-97f8-94df-6e4aa0fea1ac, Status code: 429, error code: Throttling.RateQuota, error message: Requests rate limit exceeded, please try again later.


In [ ]:
data[0]

In [ ]:
data = read_file('data/round1_train_data.jsonl')

In [ ]:
    data = []
    with open('result/qwen.jsonl') as reader:
        for line in reader:
            sample = json.loads(line)
            data.append(sample)

In [ ]:
data[0]['questions']